In [1]:
import time

In [2]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [3]:
def get_log_time():
    return datetime.now().strftime('%H:%M:%S %d/%m/%Y')

In [4]:
# ruta de carpera donde estan las llaves
print(get_log_time(), '> Estableciento conexion a base de datos...')
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud
print(get_log_time(), '> Base de datos conectado...!!!')

09:44:04 01/06/2023 > Estableciento conexion a base de datos...


09:44:12 01/06/2023 > Base de datos conectado...!!!


In [5]:
fecha = '2023-05-31'
print(get_log_time() , '> Buscando nuevos registros...')
query_dia = {'query': 'select id, canhero from dataset_315614 where date(fecha_de_registro) = \'{fech}\' order by id'.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
select_dia = amigocloud.get(project, query_dia)
data_dia = select_dia['data']
print(get_log_time(), '> Nuevos registros encontrados:', len(data_dia))

09:44:15 01/06/2023 > Buscando nuevos registros...
09:44:15 01/06/2023 > Nuevos registros encontrados: 1


In [6]:
data_dia

[{'id': 110, 'canhero': '13012 / SALAZAR ARNEZ LIDER '}]

In [7]:
for reg in data_dia:
    print(get_log_time(), 'Inicio generar reporte: Eval. Preventivo de cosechadoras:', data_dia)
    id_insp = reg['id']
    # DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_315614 where id={id_insp}'.format(id_insp=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]

    # convertir fecha a formato dd/mm/aaaa
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

    # los campos booleanos tiene formacto 'num_num', esto no es admitido para convertir a objeto
    # se modifica el dict para agregar 'k_' al inicio de todas llaves
    data = {'k_'+key : value for (key, value) in data.items()}
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # FOTOS
    query_fotos = {'query': 'select gal.s3_filename, fot.obs from dataset_315689 fot\
                                inner join gallery_48937 gal on fot.amigo_id=gal.source_amigo_id\
                                where fot.insp_ref_fot=\'{id}\''.format(id=insp.k_amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    # CREAR REPORTE
    doc = DocxTemplate(ruta + '/templates/tpl_insp_cosechadoras-preventivo.docx')
    #descargar fotos y generar lista InlineImage
    print(get_log_time(), 'Descargando imagenes...')
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['s3_filename'], width=Mm(80)),
            'obs': foto['obs']
        })
    context = {'insp':insp, 'fotos':lista_fotos_inline}
    #context = {'insp':insp}
    doc.render(context)
    # formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
    cod_nom = insp.k_canhero.split(' / ')
    file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)
    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(ruta + r'/informes/_' + file_name + '.docx')
    print(get_log_time(), 'Reporte creado: _', file_name)
print('-------------------FIN')

09:44:21 01/06/2023 Inicio generar reporte: Eval. Preventivo de cosechadoras: [{'id': 110, 'canhero': '13012 / SALAZAR ARNEZ LIDER '}]
09:44:22 01/06/2023 Descargando imagenes...
09:44:59 01/06/2023 Reporte creado: _ 13012_ECP_31-05-2023_SALAZAR ARNEZ LIDER _110
-------------------FIN


## Convertir a PDF

In [25]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 2 INFORMES
2 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.docx']

In [26]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\UTEA\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


## SUBIR A DRIVE

In [8]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [9]:
contenido = os.listdir(ruta + r'//pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    cod = split[-1].split('.')
    dic_files[int(cod[0])] = file
print(len(dic_files), 'informes')
dic_files

1 informes


{110: '13012_ECP_31-05-2023_SALAZAR ARNEZ LIDER _110.pdf'}

In [10]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "EVAL_MANTENIMIENTO_PREVENTIVO"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta + r'//pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 13012_ECP_31-05-2023_SALAZAR ARNEZ LIDER _110.pdf with mimeType application/pdf
end


In [11]:
dic_urls

{110: 'https://drive.google.com/file/d/1E34b7iPuCyvx8tUYSGiLl_z8To4_c0PD/view?usp=drivesdk'}

In [12]:
for key in dic_urls:
    query = {'query': 'update dataset_315614 set link_reporte = \'{link}\' where id = {cod}'.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32920/sql', query)
    print(key)
print('end')

110
end


In [25]:
def get_new_data():
    print(get_log_time() , '> Buscando nuevos registros...')
    query_dia = {'query': 'select id, canhero from dataset_315614 where link_reporte = \'\' or link_reporte is null'}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select_dia = amigocloud.get(project, query_dia)
    data_dia = select_dia['data']
    print(get_log_time(), '> Nuevos registros encontrados:', len(data_dia))

In [26]:
get_new_data()

10:47:56 29/05/2023 > Buscando nuevos registros...
10:48:26 29/05/2023 > Nuevos registros encontrados: 1


In [ ]:
while True:
    
    time.sleep()